### Medical procedures by physician, Medicare data (2012)
Janos Perge, 07/21/2016

Purpose:   
1) Access Medicare data on the number of performed medical procedures per physician (as a proxi for quality of care).   
2) Convert HCPCS (or CPT) procedure codes to CCS codes for further analysis  
    
Data is a public use file downloadable from CMS:
https://www.cms.gov/apps/ama/license.asp?file=http://download.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data/Downloads/Medicare_Provider_Util_Payment_PUF_CY2012_update.zip

To run this code, first visit the above url, accept CMS disclaimer, download and unzip file (2GB) and place it in CMS directory (or a location of your choice). Opening and saving file takes about 1 min on my Win10 machine with core i7 and 8GB RAM. 



#### Obtain packages

In [2]:
packageList = c("jsonlite", "data.table", "parallel","foreach","stringr","ggplot2","reshape",'plyr','rjson')

is_installed <- function(mypkg) is.element(mypkg, installed.packages()[,1])

load_or_install<-function(package_names)
{
  for(package_name in package_names)
  {
    if(!is_installed(package_name))
    {
      install.packages(package_name,repos="http://lib.stat.cmu.edu/R/CRAN")
    }
    options(java.parameters = "-Xmx8g")
    library(package_name,character.only=TRUE,quietly=TRUE,verbose=FALSE)
  }
}

load_or_install(packageList)


Attaching package: 'reshape'

The following object is masked from 'package:data.table':

    melt


Attaching package: 'plyr'

The following objects are masked from 'package:reshape':

    rename, round_any


Attaching package: 'rjson'

The following objects are masked from 'package:jsonlite':

    fromJSON, toJSON



#### 1. Access data

In [3]:
setwd("C:/Users/bubuk/Documents/R/medicare-analysis")
physician_medicare = "CMS/Medicare_Provider_Util_Payment_PUF_CY2012.txt" #available data for years 2012,2013,2014
my_data_file = "procedures2012.RData"

In [4]:
#open data from tabular file saved on HD or from Rdatafile:
if(file.exists(my_data_file) && !exists("pm")){
  load(my_data_file)
} else if(!file.exists(my_data_file)) {
  pm = read.delim(physician_medicare, stringsAsFactors=FALSE)
  pm = pm[2:nrow(pm),]
  colnames(pm) = tolower(colnames(pm))

  save(pm, file=my_data_file)
}

#### Explore data frame

In [6]:
head(pm)

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,...,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_medicare_allowed_amt,stdev_medicare_allowed_amt,average_submitted_chrg_amt,stdev_submitted_chrg_amt,average_medicare_payment_amt,stdev_medicare_payment_amt
2,1003000126,ENKESHAFI,ARDALAN,,M.D.,M,I,900 SETON DR,,CUMBERLAND,...,N,115,112,115,135.25,0,199,0,108.11565217,0.9005883395
3,1003000126,ENKESHAFI,ARDALAN,,M.D.,M,I,900 SETON DR,,CUMBERLAND,...,N,93,88,93,198.59,0,291,9.5916630466,158.87,0
4,1003000126,ENKESHAFI,ARDALAN,,M.D.,M,I,900 SETON DR,,CUMBERLAND,...,N,111,83,111,38.75,0,58,0,30.720720721,2.9291057922
5,1003000126,ENKESHAFI,ARDALAN,,M.D.,M,I,900 SETON DR,,CUMBERLAND,...,N,544,295,544,70.95,0,105,0,56.655661765,2.4313271619
6,1003000126,ENKESHAFI,ARDALAN,,M.D.,M,I,900 SETON DR,,CUMBERLAND,...,N,75,55,75,101.74,0,150,0,81.39,0
7,1003000126,ENKESHAFI,ARDALAN,,M.D.,M,I,900 SETON DR,,CUMBERLAND,...,N,95,95,95,71.06,0,104,0,55.768842105,7.4155003896


In [7]:
colnames(pm)

[1] "npi"                              "nppes_provider_last_org_name"    
 [3] "nppes_provider_first_name"        "nppes_provider_mi"               
 [5] "nppes_credentials"                "nppes_provider_gender"           
 [7] "nppes_entity_code"                "nppes_provider_street1"          
 [9] "nppes_provider_street2"           "nppes_provider_city"             
[11] "nppes_provider_zip"               "nppes_provider_state"            
[13] "nppes_provider_country"           "provider_type"                   
[15] "medicare_participation_indicator" "place_of_service"                
[17] "hcpcs_code"                       "hcpcs_description"               
[19] "hcpcs_drug_indicator"             "line_srvc_cnt"                   
[21] "bene_unique_cnt"                  "bene_day_srvc_cnt"               
[23] "average_medicare_allowed_amt"     "stdev_medicare_allowed_amt"      
[25] "average_submitted_chrg_amt"       "stdev_submitted_chrg_amt"        
[27] "average_medicare_payment_amt"     "stdev_medicare_payment_amt"

In [26]:
descriptor_vars = c("npi", "nppes_provider_last_org_name", "nppes_provider_first_name", "nppes_provider_mi", 
                    "nppes_credentials", "nppes_provider_gender", "nppes_entity_code", "nppes_provider_street1", 
                    "nppes_provider_street2", "nppes_provider_city", "nppes_provider_zip", "nppes_provider_state", 
                    "nppes_provider_country", "provider_type", "medicare_participation_indicator", "place_of_service")

payment_vars = c("npi", "hcpcs_code", "line_srvc_cnt", "bene_unique_cnt", "bene_day_srvc_cnt", 
                "average_medicare_allowed_amt", "stdev_medicare_allowed_amt", "average_submitted_chrg_amt", 
                "stdev_submitted_chrg_amt", "average_medicare_payment_amt", "stdev_medicare_payment_amt")

In [8]:
sel_pm = pm[, payment_vars]
sel_pm = data.table(sel_pm)
setkey(sel_pm, "npi") 

In [9]:
#coarse measures on payment and patient numbers:
phys_summ = sel_pm[
  , 
  list(
    service_total=sum(line_srvc_cnt),
    ben_total=sum(bene_unique_cnt),
    payment=sum(average_medicare_payment_amt * line_srvc_cnt),
    charged=sum(average_submitted_chrg_amt * line_srvc_cnt),
    allowed=sum(average_medicare_allowed_amt * line_srvc_cnt),
    unique_services_per_patient=sum(bene_day_srvc_cnt)/sum(bene_unique_cnt),
    duplicates_per_service=sum(line_srvc_cnt)/sum(bene_day_srvc_cnt),
    services_per_patient=sum(line_srvc_cnt)/sum(bene_unique_cnt)
  ),
  by="npi"
  ]

In [10]:
head(phys_summ)

,npi,service_total,ben_total,payment,charged,allowed,unique_services_per_patient,duplicates_per_service,services_per_patient
1,1.003000e+09,1.224000e+03,9.130000e+02,8.888709e+04,1.638590e+05,1.113588e+05,1.340635e+00,1.000000e+00,1.340635e+00
2,1.003000e+09,7.673000e+03,4.464000e+03,2.210911e+05,1.249454e+06,2.867675e+05,1.181900e+00,1.454321e+00,1.718862e+00
3,1.003000e+09,5.200000e+01,4.500000e+01,4.760960e+03,8.728000e+03,6.077120e+03,1.155556e+00,1.000000e+00,1.155556e+00
4,1.003000e+09,8.340000e+02,1.130000e+02,1.641112e+04,4.091400e+04,2.112492e+04,5.584071e+00,1.321712e+00,7.380531e+00
5,1.003000e+09,2.471000e+03,1.369000e+03,1.859865e+05,2.971400e+05,2.326585e+05,1.804237e+00,1.000405e+00,1.804967e+00
6,1.003000e+09,1.010000e+02,9.600000e+01,4.075150e+03,7.073000e+03,4.807060e+03,1.052083e+00,1.000000e+00,1.052083e+00


#### 2. HCPCS/CPT code to CCS conversion

In [5]:
df = pm[, c('hcpcs_code', 'hcpcs_description')]
df = df[!duplicated(df$hcpcs_code),]
# write.csv(df, file = "cptlist12.csv", row.names=FALSE, na="")
# df = read.csv("cptlist12.csv")
df$ccs_code = 0
df$ccs_desc = 'none'
df = arrange(df, hcpcs_code)

head(df)

,hcpcs_code,hcpcs_description,ccs_code,ccs_desc
1,00100,Anesthesia for procedure on salivary gland with biopsy,0,none
2,00102,Anesthesia for procedure to repair lip defect present at birth,0,none
3,00103,Anesthesia for procedure on eyelid,0,none
4,00104,Anesthesia for electric shock treatment,0,none
5,00120,Anesthesia for biopsy of external middle and inner ear,0,none
6,00126,Anesthesia for incision of ear drum,0,none


In [6]:
table_file = '2016_ccs_services_procedures.csv'
ccs_table = read.csv(table_file)
ccs_table$Code.Range <- as.character(ccs_table$Code.Range)
head(ccs_table)

,Code.Range,CCS,CCS.Label
1,'61000-61055',1,Incision and excision of CNS
2,'61105-61106',1,Incision and excision of CNS
3,'61108-61130',1,Incision and excision of CNS
4,'61150-61156',1,Incision and excision of CNS
5,'61250-61315',1,Incision and excision of CNS
6,'61320-61323',1,Incision and excision of CNS


In [7]:
#create an incremental sequence of CSS codes using Code.Range:
get_code_range <- function(inp){    
    code_range <- vector(mode="numeric", length=0) #empty 
    
    aaa = unlist(strsplit(inp, "-", fixed = TRUE))    
    aaa = sub("\'", "", aaa)
    
    if (!grepl("[a-zA-Z]", aaa[1])){  #if code does not contain letters
        aaa = as.numeric(aaa)
        code_range = seq.int(aaa[1],aaa[2])
        code_range = sprintf("%05d", code_range) # fixed width of five characters with leading zeros
        code_range = as.character(code_range)
        
    } else #if hcpcs code is alphanumeric, with the numeric part as an incremental sequence
    {
        bbb = substring(aaa[1], seq(1,nchar(aaa[1])), seq(1,nchar(aaa[1]),1)) #break up string to individual characters
        letterPos = grep("[a-zA-Z]", bbb, value = FALSE)
        letterChar = grep("[a-zA-Z]", bbb, value = TRUE)

        numericPart1 = grep("[0-9]", bbb, value = TRUE)
        numericPart1 = as.numeric(paste(numericPart1, collapse=""))

        bbb = substring(aaa[2], seq(1,nchar(aaa[2])), seq(1,nchar(aaa[2]),1)) 
        numericPart2 = grep("[0-9]", bbb, value = TRUE)
        numericPart2 = as.numeric(paste(numericPart2, collapse=""))

        cr  = seq.int(numericPart1,numericPart2)
        
        if (letterPos==1){
            code_range = sprintf("%s%04d", letterChar, cr) # fixed width of four characters with leading zeros
        } else
        {
            code_range = sprintf("%04d%s", cr, letterChar)
        }            
    }
    code_range
}

In [8]:
#find corresponding css code and description for each hcpcs code:
for (i in 1:nrow(ccs_table)){
    codeRange = get_code_range(ccs_table$Code.Range[i])
    if(length(codeRange) != 0) {
        df$ccs_code[(df$hcpcs_code %in% codeRange * 1)>0] = ccs_table$CCS[i]
        df$ccs_desc[(df$hcpcs_code %in% codeRange * 1)>0] = as.character(ccs_table$CCS.Label[i])
    }
}
df = arrange(df, hcpcs_code)
df$hcpcs_code = as.character(df$hcpcs_code)
head(df)

,hcpcs_code,hcpcs_description,ccs_code,ccs_desc
1,00100,Anesthesia for procedure on salivary gland with biopsy,232,Anesthesia
2,00102,Anesthesia for procedure to repair lip defect present at birth,232,Anesthesia
3,00103,Anesthesia for procedure on eyelid,232,Anesthesia
4,00104,Anesthesia for electric shock treatment,232,Anesthesia
5,00120,Anesthesia for biopsy of external middle and inner ear,232,Anesthesia
6,00126,Anesthesia for incision of ear drum,232,Anesthesia


In [9]:
#matching efficiency:
sum(df$ccs_code>0)/nrow(df)

[1] 0.9976467

In [11]:
#save conversion table as a csv file:
df$hcpcs_code = sprintf("%05s", df$hcpcs_code) # fixed width with leading zeros
write.csv(df, file = "CPT_to_CCS_conversion.csv", row.names=FALSE, na="")
#potential error: .csv file saves numeric hcpcs codes as an arabic number e.g. '102', ommiting zero characters in the beginning.
#This would cause mismatches later when mapping fixed-length character arrays such as '00102'. 
#Need to fix this: what are other ways of forcing write.csv to save as a fixed-length character array?

#Currently not as pressing, since R's native data format saves the codes correctly: 
save(df, file='CPT_to_CSS_Conv_Table.RData')

In [101]:
##These HCPCS codes don't have a match in the CCS table:
subset(df, ccs_code==0)

,hcpcs_code,hcpcs_description,ccs_code,ccs_desc
5370,D9940,"Occlusal guards, by report",0,none
5479,G9001,"Coordinated care fee, initial rate",0,none
5480,G9002,"Coordinated care fee, maintenance rate",0,none
5481,G9003,"Coordinated care fee, risk adjusted high, initial",0,none
5482,G9005,"Coordinated care fee, risk adjusted maintenance",0,none
5483,G9008,"Coordinated care fee, physician coordinated care oversight services",0,none
5484,G9009,"Coordinated care fee, risk adjusted maintenance, level 3",0,none
5485,G9010,"Coordinated care fee, risk adjusted maintenance, level 4",0,none
5488,G9148,Medical home level 1,0,none
5489,G9149,Medical home level ii,0,none


In [12]:
# Merge CCS codes into physician data frame:
toAppend = df
toAppend = toAppend[,-2] ## drop hcpcs description (redundant with other dataframe)

#pm = merge(pm, toAppend, by='hcpcs_code') #runs out of memory, either if used on pm or on a subset of pm. Only works with data.Table!

In [23]:
pm = data.table(pm)
setkey(pm, "hcpcs_code") 
pm = merge(pm, toAppend, by='hcpcs_code')

In [24]:
head(pm)

,hcpcs_code,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,...,bene_unique_cnt,bene_day_srvc_cnt,average_medicare_allowed_amt,stdev_medicare_allowed_amt,average_submitted_chrg_amt,stdev_submitted_chrg_amt,average_medicare_payment_amt,stdev_medicare_payment_amt,ccs_code,ccs_desc
1,00100,1396897104,RIDGLEY,PHILLIP,R,CRNA,M,I,171 ASHLEY AVE,,...,12,14,125.05928571,27.539691217,1401.4285714,305.13643653,100.04857143,22.032023076,232,Anesthesia
2,00100,1508884156,FUREY,ERIN,J,MD,F,I,11100 EUCLID AVE,,...,12,12,353.00416667,80.615788513,2122.5,652.5862012,282.4025,64.492407838,232,Anesthesia
3,00100,1528032067,PLYLER,JANICE,M,CRNA,F,I,107 PLANTATION CIR,,...,14,14,185.57,27.338754805,572.92857143,84.316126475,148.45642857,21.87183205,232,Anesthesia
4,00100,1780646562,KLEIN,KEVIN,W,MD,M,I,5323 HARRY HINES BLVD,,...,13,13,357.89923077,198.10791711,2098.4615385,965.20899414,286.32076923,158.48520443,232,Anesthesia
5,00102,1700846235,DANGELO,SCOTT,A,CRNA,M,I,555 E CHEVES ST,,...,17,17,200.42941176,52.47849223,1569.7058824,827.42597072,152.30705882,43.448522044,232,Anesthesia
6,00103,1003019191,WARREN,DAVID,T,M.D.,M,I,1108 ROSS CLARK CIR,,...,14,14,90.253571429,15.035313008,393.53714286,229.38057009,65.611428571,13.137521976,232,Anesthesia


In [25]:
my_data_file2 = "procedures2012_with_CCS.RData"
save(pm, file=my_data_file2)